 <div style="text-align:center;">
   <span style="color:green; font-size:larger; font-weight:bold;">Vérification de la loi d'Okun dans plusieurs pays du monde</span><br><br>
  <span style="font-weight:bold;">Présenté par:</span><br>
  <span>NOUBOUSSI GNINTEDEM LUCIE MARIMAR</span><br>
  <span>YOUSRA JEDDOUB</span> <br>
  <span>AMINA MANSEUR</span>
</div>


# <span style="color:green">Introduction</span><br><br> 

Dans cette première partie, l'objectif est de construire les DataFrames nécessaires à notre projet.<br>
La démarche suivie est la suivante :<br>
<div style="margin-left: 20px;">
    <span style="font-weight:bold;">1.</span> Importer les différentes BD en local.<br>
    <span style="font-weight:bold;">2.</span> Construire les Dataframes par l'utilisation des méthodes du module Pandas.<br>
    <span style="font-weight:bold;">3.</span> Les bases de données étant disponibles en ligne ; Automatisation de l'importation en utilisant un url (ceci garantit que les données soient toujours à jour).<br>
    <span style="font-weight:bold;">4.</span> Optimisation du code de facon à ce qu'il soit le plus reproductible possible et ce en définissant des fonctions selon les besoins du projet.
    </div>

# <span style="color:green">I- Importation et installation des packages</span>

Pour plus de clarté et de lisibilité du code, nous déclarons l'ensemble des imports nécessaires dans un fichier distinct "declaration.py". Ce dernier est ainsi appelé au tout début. Si des ajouts, suppressions ou mises à jour des dépendances sont requises, ces dernières sont faites simplement dans le fichier "declaration.py".

In [ ]:
from importlib import reload
import declarations as d
reload(d)

import pycodestyle as pep8 # not used
import zipfile
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pycountry
import io
from io import BytesIO
import seaborn as sns
import missingno as msno
import statistics 


# <span style="color:green; ">II- Construction de la base de données</span>

## <span style="color:green; text-align:center;">II-1 Importation de la base GemDataEXTR.Zip</span>

L'importation se fait via l'url de téléchargement.

In [ ]:
url = "https://datacatalogfiles.worldbank.org/ddh-published/0037798/DR0092042/GemDataEXTR.zip?"
d.load(url, "GemDataEXTR.zip")

In [ ]:
# Choix des fichiers à extraire
nom_fichiers = [('Unemployment Rate, seas. adj..xlsx', 'monthly'),
              ('GDP Deflator at Market Prices, LCU.xlsx', 'quarterly')
]

# Extraction des bases dans la mémoire: chomage et PIB déflaté
df_Unemployement, df_GDP = [d.extraire_fichier_zip('GemDataEXTR.zip', nom_fichier, nom_feuille)
                           for nom_fichier, nom_feuille in nom_fichiers]

<div style="margin-left: 20px;">
-L'extraction des données sur le taux de chômage et le taux de croissance du PIB déflaté par pays est désormais complète. <br>
-Dans la prochaine étape, nous procéderons à une exploration rapide de ces données et les fusionnerons pour une analyse plus approfondie.
</div>


### <span style="color:green; text-align:center;">II-1-1 Préparation des bases de données</span>

#### <span style="color:green; text-align:center;">II-1-1-1 Base taux de chômage</span>

In [ ]:
# Visualisation
df_Unemployement.head()

<div style="margin-left: 20px;">
-La 1ère ligne de la base 'df_Unemployement' est vide.<br>
-Ainsi on souhaite commencer à partir de l'année 1994, la ligne correspondant au dernier mois de l'année 1993 sera supprimée.
</div>

In [ ]:
# Suppression des deux premières lignes
df_Unemployement = df_Unemployement.iloc[2:].copy()

In [ ]:
# Informations sur la DF (nombre de valeurs non nulles, type de données de chaque colonne...)
df_Unemployement.info()

Le type de chaque variable est approprié et correspond aux types attendus. Il s'agit d'un nombre flottant de 64 bits.

In [ ]:
# Taille de la DF 
print( "La DataFrame est de dimension", df_Unemployement.shape[0], "lignes et", df_Unemployement.shape[1], "colonnes.")


##### Détection des doublons

In [ ]:
# Vérification des doublons
print("Nombre total de doublons dans df_Unemployement :",
      df_Unemployement[df_Unemployement.duplicated()].shape[0])

In [ ]:
# Supprimer les doublons (s'ils existent)
df_Unemployement.drop_duplicates(inplace=True)

print("Nombre total de doublons dans df_Unemployement :", 
      df_Unemployement[df_Unemployement.duplicated()].shape[0])

##### Correction des noms des pays

Pour rendre la DF plus lisible, on exprime les noms des pays par leurs codes ISO 3166-1 alpha-3, soit des abréviations à trois (3) lettres.

In [ ]:
"""# Appliquer la correction sur chaque colonne du DataFrame
df_Unemployement.columns = d.correct_country_name(df_Unemployement.columns)

# Listes des pays détectés
pays = d.detect_country_name(df_Unemployement.columns)

# Base avec colonnes corrigées
df_Unemployement = df_Unemployement[pays]"""

##### Détection des valeurs manquantes

Pour rendre la DF plus lisible, on exprime les noms des pays par leurs codes ISO 3166-1 alpha-3, soit des abréviations à trois (3) lettres.

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_Unemployement)

Les données consistent en des séries temporelles. Chaque pays disposant de sa propre série temporelle du taux de chômage.<br>
D'après la visualisation ci-dessus, on constate que pour la plupart des pays : 
<div style="margin-left: 20px;">
-Les valeurs manquantes sont en début de la période considérée.<br></div>

Ainsi, nous allons garder uniquement les pays avec au moins 60% d'observations non manquantes.</div>


In [ ]:
# Suppression des pays avec au moins 40% de valeurs manquantes sur la période
df_Unemployement = d.missing(df_Unemployement)

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_Unemployement)

In [ ]:
# Taille de la DF 
print( "La DataFrame est de dimension", df_Unemployement.shape[0], "lignes et", df_Unemployement.shape[1], "colonnes.")

À ce stade, on retrouve une DF où uniquement les pays avec suffisamment de données sont representés. 

##### Imputation des valeurs manquantes

L'imputation par la médiane étant plus robuste aux valeurs aberrantes sera privilégiée ici.

In [ ]:
# Imputation des valeurs manquantes 
df_Unemployement = d.fill_missing_with_median(df_Unemployement)

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_Unemployement)

Les valeurs manquantes sont totalement imputées. On obtient une DataFrame complète et propre prête à être utilisée pour l'analyse ou la modélisation. 

In [ ]:
# Visualisation
df_Unemployement.head()

In [ ]:
# Colonnes présentes dans Uemploy
pays = df_Unemployement.columns
print('Les pays de la DataFrame  df_Unemployement sont :',pays)

In [ ]:
# L'index de la DF
df_Unemployement.index

**Précision :** <br>

À ce stade :
<div style="margin-left: 20px;">
-La DataFrame "df_Unemployement" est un tableau à 358 lignes et 31 colonnes. Nous disposons alors d'une série temporelle du taux de chômage de taille 358 observations pour chaque pays parmi les 31 pays.<br>
-L'index de la DF est de type date. Il s'agit en effet de données mensuelles s'étalant sur la période allant de Janvier 1994 à Janvier 2023.</div>

Pour faciliter le recours à la DataFrame "df_Unemployement", on extrait cette dernière sous un format **.csv** .De cette manière, on pourrait y recourir à partir de  n'importe quel fichier.

In [ ]:
# Extraction de la DF format .csv
df_Unemployement.to_csv('Unemployement_rates.csv', index=True)

##### Transformation des données mensuelles en données trimestrielles

Plus loin, nous aurrons à réaliser une jointure entre la DataFrame comportant les taux de chomage et celle contenant les taux de croissance du PIB. Les données dont on dispose sur les taux de croissance du PIB sont trimestrielles. Il est alors plus cohérent de transformer les données mensuelles en données trimestrielles pour la DataFrame "df_Unemployement".

In [ ]:
# Grouper par année de 12 mois chacun
df_Unemployement = df_Unemployement\
    .groupby(df_Unemployement.index.year)\
        .filter(lambda x: len(x) == 12)
df_Unemployement = d.pd.DataFrame(df_Unemployement)

L'idée du code précedent est de ne garder que les années pour lequelles les taux de chômage des 12 mois de l'année sont renseignés. L'année 2023, par exemple, sera éliminée car nous disposons que du taux de chomage du 1er mois de cette année.<br>

Le regroupement va se faire par une moyenne arithmétique simple.


In [ ]:
# Transformation
df_Unemployement = df_Unemployement.resample('Q-JAN').mean()
# Ignorer les jours dans l'index
df_Unemployement.index = df_Unemployement.index.strftime('%Y-%m')

In [ ]:
# Taille de la DF 
print( "La DataFrame est de dimension", df_Unemployement.shape[0], "lignes et", df_Unemployement.shape[1], "colonnes.")

#### <span style="color:green; text-align:center;">II-1-1-2 Dataframe du taux de croissance du PIB</span>

In [ ]:
# Visualisation
df_GDP.head(5)

La 1ère ligne étant vierge. Cette dernière sera supprimée.

In [ ]:
# Supprimez la ligne avec l'index NaN du DataFrame
df_GDP = df_GDP.drop(df_GDP.index[0])

# L'index de la DF
df_GDP.index

L'index de la DataFrame est de type date, les données sont trimestrielles et s'étalent sur la période allant du premier trimestre de 1994 au dernier trimestre de 2023.

In [ ]:
# Informations sur la DF (nombre de valeurs non nulles, type de données de chaque colonne...)
df_GDP.info()

Le type de données est conforme aux attentes.

In [ ]:
# Taille de la DF 
print( "La DataFrame est de dimension", df_GDP.shape[0], "lignes et", df_GDP.shape[1], "colonnes.")

In [ ]:
# Formater l'index pour obtenir '1994-01' au lieu de '1994-01-01'
df_GDP.index = df_GDP.index.strftime('%Y-%m')

##### Correction des noms des pays

Comme pour la DF "df_unemployment", nous allons remplacer les noms des pays par leurs codes ISO.

In [ ]:
# Appliquer la correction sur chaque colonne du DataFrame
df_GDP.columns = d.correct_country_name(df_GDP.columns)

# Base avec colonnes corrigées presente dans Unemploy
df_GDP=df_GDP[pays]

##### Détection des doublons

In [ ]:
# Vérifier la présence de doublons
print("Nombre total de doublons dans df_GDP :", 
      df_GDP[df_GDP.duplicated()].shape[0])

In [ ]:
# Supprimer les doublons (s'ils existent)
df_GDP.drop_duplicates(inplace=True)

print("Nombre total de doublons dans df_GDP :", 
      df_GDP[df_GDP.duplicated()].shape[0])

##### Détection des valeurs manquantes

Les valeurs manquantes sont visualisées via un diagramme à barre et un heatmap.

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_GDP)

Les données consistent en des séries temporelles. Chaque pays disposant de sa propre série temporelle du taux de croissance du PIB.<br>
D'après la visualisation ci-dessus, on constate que pour la plupart des pays : 
<div style="margin-left: 20px;">
-Les valeurs manquantes sont en début de la période considérée.<br></div>

Ainsi, nous allons garder uniquement les pays avec au moins 60% d'observations non manquantes.</div>

In [ ]:
# Suppression des pays avec au moins 90% des valeurs manquantes sur la période
df_GDP = d.missing(df_GDP)

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_GDP)

##### Imputation des valeurs manquantes

On considère la médiane comme méthode d'imputation.

In [ ]:
# Correction des valeurs manquantes 
df_GDP = d.fill_missing_with_median(df_GDP)

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_GDP)

Les valeurs manquantes sont totalement imputées. On obtient une DataFrame complète et propre prête à être utilisée pour l'analyse ou la modélisation. 

In [ ]:
# Taille de la DF 
print( "La DataFrame est de dimension", df_GDP.shape[0], "lignes et", df_GDP.shape[1], "colonnes.")

In [ ]:
# Colonnes présentes dans gdp
pays1 = df_GDP.columns
print('Les pays de la DataFrame  df_Unemployement sont :',pays1)

In [ ]:
# Colonnes non présentes dans l'une des DF et pas dans l'autre
print(list(set(pays) - set(pays1))) 

In [ ]:
# Nombre de pays dans chaque DF
print("Le nombre de pays dans 'df_Unemployement' est de", len(pays), ".")
print("Le nombre de pays dans 'df_GDP' est de", len(pays1), ".")

Connaitre le nombre de pays dans chaque DF permet d'avoir une idée sur le nombre de pays qu'on aura au final après la jointure entre 'df_GDP' et 'df_Unemployement. 

**Précision :**

À ce stade :
<div style="margin-left: 20px;">
-La DataFrame "df_GDP" est un tableau à 120 lignes et 30 colonnes. Nous disposons alors d'une série temporelle du taux de croissance du PIB de 120 observations pour chaque pays parmi les 30 pays.<br>
-L'index de la DF est de type date. Il s'agit en effet de données trimestrielles s'étalant sur la période allant du premier trimestre de 1994  jusqu'au dernier trimestre de 2023.</div>


### <span style="color:green; text-align:center;">II-1-2 Fusion des deux bases</span>

#### <span style="color:green; text-align:center;">II-1-2-1 Format long</span>

In [ ]:
reload(d)
# Tranformation des bases en format long 
dfs = d.transform(df_Unemployement, 'Unemployment_rate')
dfs1 = d.transform(df_GDP, 'GDP_rate')
dfs1.head(10)

In [ ]:
# Fusion des bases
df_merge1 = d.pd.merge(dfs, dfs1, on=['YEAR', 'COUNTRY'], how='left') # La clé de jointure est composée des dates et des pays
df_merge1.head()

Suite à la jointure, on obtient une DF contenant en colonne les dates trimestrielles, les pays, les taux de chômage et les taux de croissance du PIB.

#### <span style="color:green; text-align:center;">II-1-2-2 Format large </span>

On souhaite avoir une DF où en lignes nous avons les dates, et en colonnes nous avons pour chaque pays une colonne pour le taux de chomage et une pour le taux de croissance du PIB de manière successive.

In [ ]:
# Jointure des DF df_Unemployement et df_GDP
quart_data = pd.merge(df_Unemployement, df_GDP, left_index=True, right_index=True, how='inner' )

# Supprimer le suffixe '_x' et le remplacer par 'rate', de meme supprimer le suffixe '_y' et le remplacer par 'gdp'
new_columns = [col.replace('_x', '_rate').replace('_y', '_gdp') for col in quart_data.columns]

# Renommer les colonnes de la DataFrame
quart_data.columns = new_columns

# Trier les colonnes de la DataFrame de facon à placer pour chaque pays le PIB et taux de chomage l'un après l'autre 
quart_data= quart_data.reindex(sorted(quart_data.columns), axis=1)

# Renommer l'index
quart_data = quart_data.rename_axis("Dates")

In [ ]:
# Visualisation
quart_data.head(5)

In [ ]:
# Taille de la DF 
print( "La DataFrame est de dimension", quart_data.shape[0], "lignes et", quart_data.shape[1], "colonnes.")

In [ ]:
# Exportation de la DF construite sous un fichier .csv
quart_data.to_csv("quart_data.csv")

In [ ]:
df_Unemployement.index = pd.to_datetime(df_Unemployement.index)
df_GDP.index = pd.to_datetime(df_GDP.index)

In [ ]:
### 3.3. Jointure entre taux de chomage et PIB moyen sur toute la période
mean_rate = d.pd.DataFrame(df_Unemployement.mean(), columns=['Taux de chomage'])
mean_gdp = d.pd.DataFrame(df_GDP.mean(), columns=['PIB'])
mean_data= d.pd.merge(mean_rate, mean_gdp, left_index=True, right_index=True, how='inner')

## <span style="color:green; text-align:center;">II-2 Importation de la base HNP_Stats_EXCEL.Zip via son url de téléchargement</span>

In [ ]:
url = "https://databank.worldbank.org/data/download/HNP_Stats_EXCEL.zip"
d.load(url,"HNP_Stats_EXCEL.zip")

In [ ]:
# Choix des fichiers à extraire
nom_fichiers = [('HNP_StatsEXCEL.xlsx', 'Data')]

# Extraction des bases dans la mémoire: chomage et PIB déflaté
Big_data = [d.extraire_fichier_zip('HNP_Stats_EXCEL.zip', nom_fichier, nom_feuille)
                           for nom_fichier, nom_feuille in nom_fichiers]

L'extraction des données sur le niveau d'éducation, l'espérance de vie, et le taux de croissance de la population. <br>
Dans la prochaine étape, nous procéderons à une exploration rapide de ces données et les fusionnerons pour une analyse plus approfondie.


### <span style="color:green; text-align:center;">II-2-1 Préparation des bases avant fusion</span>

In [ ]:
Big_data = Big_data[0]
Big_data.head()


Le type de chaque variable est approprié et correspond aux types attendus.

#### <span style="color:green; text-align:center;">II-2-1-1 Base espérance de vie</span>

In [ ]:
# Extraction des données sur l'espérance de vie 
df_LE = d.extract2(Big_data, 'expectancy','total')
df_LE.head(2)

In [ ]:
# Construction de la base life expentancy
reload(d)
df_LE = d.treat_info(df_LE, pays1)
df_LE.head(2)

##### Detections des valeurs manquantes

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_LE)

print("Nombre total de valeurs manquantes est de ", 
      d.missing_plot(df_LE))

On n'observe aucune valeur manquante dans la base

#### <span style="color:green; text-align:center;">II-2-1-2 Base taux de croissance démographique</span>

In [ ]:
# Extraction des donnés sur la croissance démographique 
reload(d)
df_pop = d.extract2(Big_data, '^Population growth \(annual %\)$','')
df_pop.head(2)

In [ ]:
# Informations sur la DF (nombre de valeurs non nulles, type de données de chaque colonne...)
df_pop.info()

In [ ]:
df_pop.head()

In [ ]:
# Construction de la base life expentancy
reload(d)
df_pop = d.treat_info(df_pop, pays1)
df_pop.tail()

##### Détection des valeurs manquantes

In [ ]:
# Plot des valeurs manquantes
d.missing_plot(df_pop)

on observe aucune valeur manquantes dans la base

### <span style="color:green; text-align:center;">II-2-2 Fusion des deux bases</span>

In [ ]:
reload(d)
## Tranformation des bases en format long 
dfs = d.transform(df_LE, 'life_expentancy')
dfs1 = d.transform(df_pop, 'pop_growth_rate')
dfs.head()

In [ ]:
# Fusion des bases

df_merge2 = d.pd.merge(dfs, dfs1, on=['YEAR', 'COUNTRY'], how='left')
df_merge2.head()

In [ ]:
# Fusion merge1 et merge2.

df_merge3 = d.pd.merge(df_merge1, df_merge2, on=['YEAR', 'COUNTRY'], how='left')
df_merge3.head()

In [ ]:
# Enregistrez le DataFrame au format CSV
df_merge3.to_csv('final_data.csv', index=False)

## <span style="color:green; text-align:center;">II-3 Importation de la base Spatial Inequality Database via son url de téléchargement</span>

In [ ]:
url = "https://datacatalogfiles.worldbank.org/ddh-published/0064524/DR0091539/inequality%20GMD%20World%20Bank.xlsx?versionId=2023-05-22T17:13:22.2930786Z"
d.load(url,"inequality GMD World Bank.xlsx")

In [ ]:
df_SID = d.pd.read_excel("inequality GMD World Bank.xlsx",  sheet_name='data', index_col=0)
df_SID.head()

### <span style="color:green; text-align:center;">II-3-1 Préparation des bases avant fusion</span>

#### <span style="color:green; text-align:center;">II-3-1-1 base indice Theil</span>

In [ ]:
# Base de données de l'indice Theil
df_theil = df_SID.pivot(index='year', columns='countryname', values='index')
df_theil.head(5)

In [ ]:
df_theil = df_theil.drop(columns=df_theil.columns[0])
df_theil.info()

In [ ]:
#df_theil.columns

##### Corrections des noms des pays

In [ ]:
# Appliquer la correction sur chaque colonne du DataFrame
df_theil.columns = d.correct_country_name(df_theil.columns)

# base avec colonnes corrigées contenues dans les autres bases
#df_theil=df_theil[pays1]
#on a que 19 pays

In [ ]:
# Toutes les quatres bases à extraire ici
# La moyenne de l'indice de theil sur toutes les périodes
mean_theil = theil.mean(axis=0) 
# Base de données du taux d'urbanisation
urban = gmd.pivot(index='year', columns='countryname', values='sp_urb_totl_in_zs')
urban.head(5)
# La moyenne du taux d'urbanisation sur toutes les périodes
mean_urban = urban.mean(axis=0)
# Base de données du ratio de pauvreté
poverty = gmd.pivot(index='year', columns='countryname', values='si_pov_lmic')
poverty.head(5)
# La moyenne du ratio de pauvreté sur toutes les périodes
mean_poverty = poverty.mean(axis=0)

# IV- GMD  ici on a pas les meme années que faire ??

## 1.  Importation

In [ ]:
# Gini Mean Difference
gmd = pd.read_excel("C:/Users/yousr/Downloads/Projet_py/inequality GMD World Bank.xlsx",  sheet_name='data', index_col=0)
gmd.head(5)
col_names = gmd.columns
print(col_names)

## 2. Construction de la DF

### 2.1. Extraction des colonnes nécessaires

In [ ]:
# Base de données de l'indice Theil
theil = gmd.pivot(index='year', columns='countryname', values='index')
theil.head(5)

In [ ]:
# La moyenne de l'indice de theil sur toutes les périodes
mean_theil = theil.mean(axis=0) 

In [ ]:
# Base de données du taux d'urbanisation
urban = gmd.pivot(index='year', columns='countryname', values='sp_urb_totl_in_zs')
urban.head(5)

In [ ]:
# La moyenne du taux d'urbanisation sur toutes les périodes
mean_urban = urban.mean(axis=0)

In [ ]:
# Base de données du ratio de pauvreté
poverty = gmd.pivot(index='year', columns='countryname', values='si_pov_lmic')
poverty.head(5)

In [ ]:
# La moyenne du ratio de pauvreté sur toutes les périodes
mean_poverty = poverty.mean(axis=0)

In [ ]:
# Dans pandas, une Df à une colonne correspond à une série. On donne un nom à chaque série pour une éventuelle jointure
mean_poverty.name = 'Ratio de pauvreté'
mean_urban.name = "Taux urbanisation"
mean_theil.name = 'Theil'

## 2.2. Jointure 

In [ ]:
# Jointure entre df1 et df2 sur la colonne "countryname"
merged1 = pd.merge(mean_poverty, mean_urban, on='countryname', how='inner')
# Jointure entre le résultat précédent (merged_df) et df3 sur la colonne "countryname"
final_merged1 = pd.merge(merged1, mean_theil, on='countryname', how='inner')

In [ ]:
# Dans pandas, une Df à une colonne correspond à une série. On donne un nom à chaque série pour une éventuelle jointure
mean_LE.name = 'Espérance de vie'
mean_PGR.name = "Croissance démo"

In [ ]:
final_merged1.index.name = 'Country Name'  ####################################################################################
mean_LE.index.name = 'Country Name'
mean_PGR.index.name = 'Country Name'
merged_mean.index.name = 'Country Name'

In [ ]:
# Jointure entre df1 et df2 sur la colonne "countryname"
merged2 = pd.merge(final_merged1, mean_PGR, on='Country Name', how='inner')
# Jointure entre le résultat précédent (merged_df) et df3 sur la colonne "countryname"
merged3 = pd.merge(merged2, mean_LE, on='Country Name', how='inner')

# Transposé de merged3

In [ ]:
merged3_transpose = merged3.transpose()
col_names = merged3_transpose.columns
merged3_transpose.head(5)

# Code ISO des pays

In [ ]:
# Pour faciliter la lecture de la DF, on remplace les noms des pays par leurs codes ISO  correspondant

# Dictionnaire de correspondance entre noms complets des pays et leurs codes
corresp = {country.name: country.alpha_3 for country in pycountry.countries}

# Liste initiale des noms complets des colonnes
country = col_names
# Transformation des noms complets des colonnes en abréviations
country_codes = [corresp.get(pays, pays) for pays in country]

# Remplacement des noms des pays par leurs codes dans la merged
merged3_transpose.columns = country_codes
merged3_transpose.columns
merged3_transpose.head(5)

#Ordre alphabétique des colonnes 
merged3_transpose_sort = merged3_transpose.sort_index(axis=1)
merged3_transpose_sort.head(5)

In [ ]:
merged4 = merged3_transpose.transpose()
merged4.index.name = 'Country Name'
merged4.head(5)

In [ ]:
final_merged = pd.merge(merged4, merged_mean, on='Country Name', how='inner')